In [1]:
import os
import sys
import numpy as np
import theano.tensor as T
import codecs
import theano.sandbox.cuda
theano.sandbox.cuda.use("gpu0")

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, CuDNN 4007)


In [2]:
sys.path.append('/usr1/home/ssandeep/UltraDeep/')

In [3]:
from network import LSTM
from layer import HiddenLayer, EmbeddingLayer
from learning_method import LearningMethod

In [4]:
path_to_oracle = '/usr1/home/ssandeep/IncrementalMT/oracle.tsv'

In [5]:
lines = [line.strip().split(' ||| ') for line in codecs.open(path_to_oracle, 'r', encoding='utf8')]

In [6]:
sentences = []
curr_sentence_pointer = 0
for ind, line in enumerate(lines):
    if len(line) == 1 and line[0] == '':
        sentences.append(lines[curr_sentence_pointer:ind])
        curr_sentence_pointer = ind + 1

In [7]:
soure_words = set()
for sentence in sentences:
    source_sentence = sentence[0][0]
    for word in source_sentence.split():
        soure_words.add(word)

In [8]:
target_words = set()
for sentence in sentences:
    target_sentence = sentence[1][0]
    for word in target_sentence.split():
        target_words.add(word)

In [9]:
for sentence in sentences:
    for state in sentence[2:]:
        assert len(state) == 5

In [10]:
source_word2ind = {word:ind for ind, word in enumerate(soure_words)}
source_ind2word = {ind:word for ind, word in enumerate(soure_words)}
target_word2ind = {word:ind for ind, word in enumerate(target_words)}
target_ind2word = {ind:word for ind, word in enumerate(target_words)}

In [39]:
source_word2ind['<s>'] = len(source_word2ind)
source_word2ind['</s>'] = len(source_word2ind) + 1
source_word2ind['$NONE$'] = len(source_word2ind) + 2
target_word2ind['<s>'] = len(target_word2ind)
target_word2ind['</s>'] = len(target_word2ind) + 1
target_word2ind['$NONE$'] = len(target_word2ind) + 2
source_ind2word[len(source_word2ind)] = '<s>'
source_ind2word[len(source_word2ind) + 1] = '</s>'
source_ind2word[len(source_word2ind) + 2] = '$NONE$'
source_ind2word[len(source_word2ind) + 3] = '<ss>'
source_ind2word[len(source_word2ind) + 4] = '</se>'
source_ind2word[len(source_word2ind) + 5] = '<fcs>'
source_ind2word[len(source_word2ind) + 6] = '</fce>'
target_ind2word[len(target_word2ind)] = '<s>' 
target_ind2word[len(target_word2ind) + 1] = '</s>' 
target_ind2word[len(target_word2ind) + 2] = '$NONE$'

In [22]:
len(target_word2ind)

71180

In [40]:
stack_input = T.ivector()
forward_context_input = T.ivector()
target_input = T.ivector()
action_prediction = T.scalar()
target_output = T.ivector()

In [41]:
# Generate synthetic data to test dimensions
syn_stack_input = np.random.randint(low=0, high=len(source_word2ind), size=(5,)).astype(np.int32)
syn_forward_context_input = np.random.randint(low=0, high=len(source_word2ind), size=(6,)).astype(np.int32)
syn_action_prediction = np.random.randint(low=0, high=2, size=(1,)).astype(np.float32)[0]
syn_target_input = np.random.randint(low=0, high=len(target_word2ind), size=(7,)).astype(np.int32)
syn_target_output = np.random.randint(low=0, high=len(target_word2ind), size=(7,)).astype(np.int32)

In [42]:
# Neural Inventory

src_emb_dim      = 256  # source word embedding dimension
tgt_emb_dim      = 256  # target word embedding dimension
src_lstm_hid_dim = 512  # source LSTMs hidden dimension
tgt_lstm_hid_dim = 2 * src_lstm_hid_dim  # target LSTM hidden dimension


source_embedding = EmbeddingLayer(input_dim=len(source_word2ind) + 1, output_dim=src_emb_dim)
target_embedding = EmbeddingLayer(input_dim=len(target_word2ind) + 1, output_dim=tgt_emb_dim)

stack_source_lstm_forward = LSTM(input_dim=src_emb_dim, hidden_dim=src_lstm_hid_dim, name='source_stack_lstm', with_batch=False)
stack_source_lstm_backward = LSTM(input_dim=src_emb_dim, hidden_dim=src_lstm_hid_dim, name='source_stack_lstm', with_batch=False)

forward_context_lstm_forward = LSTM(input_dim=src_emb_dim, hidden_dim=src_lstm_hid_dim, name='source_forward_context_lstm', with_batch=False)
forward_context_lstm_backward = LSTM(input_dim=src_emb_dim, hidden_dim=src_lstm_hid_dim, name='source_forward_context_lstm', with_batch=False)

target_lstm = LSTM(input_dim = 2 * tgt_emb_dim, hidden_dim=tgt_lstm_hid_dim, name='target_lstm', with_batch=False)
action_prediction_weights = HiddenLayer(input_dim=2 * stack_source_lstm_forward.hidden_dim + 2 * forward_context_lstm_forward.hidden_dim, output_dim=1)
target_word_decoding_weights = HiddenLayer(input_dim=tgt_lstm_hid_dim + 2 * src_lstm_hid_dim, output_dim=len(target_word2ind), activation='softmax')
projection_weights = HiddenLayer(input_dim = 2 * src_lstm_hid_dim, output_dim = tgt_emb_dim, activation='tanh')

action_params = source_embedding.params + stack_source_lstm_forward.params + stack_source_lstm_backward.params + forward_context_lstm_forward.params + forward_context_lstm_backward.params + action_prediction_weights.params
seq_seq_params = source_embedding.params + stack_source_lstm_forward.params + stack_source_lstm_backward.params + target_embedding.params + target_lstm.params[:-1] + projection_weights.params + target_word_decoding_weights.params

MemoryError: ('Error allocating 583106560 bytes of device memory (out of memory).', "you might consider using 'theano.shared(..., borrow=True)'")

In [ ]:
# =====================================================
# The computational graph for this method
# =====================================================

# Get the embedding matrices seq_len x embdding_dim
stack_embedding_matrix = source_embedding.link(stack_input)
forward_context_embedding_matrix = source_embedding.link(forward_context_input)

# ===========================================================
# Get LSTM representations of the stack, forward context
# ===========================================================

# Stack Representation
stack_source_lstm_forward.link(stack_embedding_matrix)
stack_source_lstm_backward.link(stack_embedding_matrix[::-1, :])
stack_lstm_representation = T.concatenate((stack_source_lstm_forward.h, stack_source_lstm_backward.h[::-1,:]), axis=1)
print 'stack_lstm_representation', stack_lstm_representation.eval({stack_input:syn_stack_input}).shape

# Forward Context Representation
forward_context_lstm_forward.link(forward_context_embedding_matrix)
forward_context_lstm_backward.link(forward_context_embedding_matrix[::-1, :])
forward_context_representation = T.concatenate((forward_context_lstm_forward.h, forward_context_lstm_backward.h[::-1,:]), axis=1)
print 'forward_context_lstm_representation', forward_context_representation.eval({forward_context_input:syn_forward_context_input}).shape

# Concatenate representations and make a prediction about what action to take
concatenated_representation = T.concatenate((stack_lstm_representation[-1], forward_context_representation[-1]))
print 'concatenated_representation', concatenated_representation.eval({stack_input:syn_stack_input, forward_context_input:syn_forward_context_input}).shape
prediction = action_prediction_weights.link(concatenated_representation).mean() # .mean() is a hack to make it a scalar (since its only a single value, it shouldn't matter)
print 'predicton', prediction.eval({stack_input:syn_stack_input, forward_context_input:syn_forward_context_input})

# Compute squared-error loss between predicted action and gold action
action_prediction_loss = ((action_prediction - prediction) ** 2).mean()
print 'action_prediction_loss', action_prediction_loss.eval({stack_input:syn_stack_input, forward_context_input:syn_forward_context_input, action_prediction:syn_action_prediction})

# Get target input embeddings
target_embeddings = target_embedding.link(target_input)
print 'target_embeddings', target_embeddings.eval({target_input:syn_target_input}).shape

target_lstm.h_0 = stack_lstm_representation[-1]
print 'target_lstm_h0', target_lstm.h_0.eval({stack_input:syn_stack_input}).shape
repeated_src_context = T.repeat(stack_lstm_representation[-1].dimshuffle('x', 0), target_embeddings.shape[0], axis=0)
print 'repeated_src_context', repeated_src_context.eval({stack_input:syn_stack_input, target_input:syn_target_input}).shape
repeated_src_context = projection_weights.link(repeated_src_context)
print 'repeated_src_context', repeated_src_context.eval({stack_input:syn_stack_input, target_input:syn_target_input}).shape
target_embeddings = T.concatenate((target_embeddings, repeated_src_context), axis=1)
print 'target_embeddings', target_embeddings.eval({stack_input:syn_stack_input, target_input:syn_target_input}).shape
target_lstm.link(target_embeddings)
print 'target_lstm', target_lstm.h.eval({stack_input:syn_stack_input, target_input:syn_target_input}).shape

transition = target_lstm.h.dot(stack_lstm_representation.transpose())
transition = transition.dot(stack_lstm_representation)
print 'transition', transition.eval({stack_input:syn_stack_input, target_input:syn_target_input}).shape

transition_last = T.concatenate([transition, target_lstm.h], axis=1)
decoded_words = target_word_decoding_weights.link(transition_last)
print 'decoded_words', decoded_words.eval({stack_input:syn_stack_input, target_input:syn_target_input}).shape
decoding_loss = T.nnet.categorical_crossentropy(decoded_words, target_output).mean()
print 'decoding_loss', decoding_loss.eval({stack_input:syn_stack_input, target_input:syn_target_input, target_output:syn_target_output})

'''
# Appened source representation to input at every step
repeated_src_context = T.repeat(stack_lstm_representation[-1].dimshuffle('x', 0), target_embeddings.shape[0], axis=0)
#repeated_src_context = proj_layer2.link(repeated_src_context)
target_embeddings = T.concatenate((target_embeddings, repeated_src_context), axis=1)

# Get the decoded sentence by connecting Encoder & Decoder
connection = projection_weights.link(stack_lstm_representation[-1])
print 'connection', connection.eval({stack_input:syn_stack_input}).shape
target_lstm.h_0 = connection
target_lstm.link(target_embeddings)
print 'target_lstm', target_lstm.h.eval({stack_input:syn_stack_input, target_input:syn_target_input}).shape

transition = target_lstm.h.dot(stack_lstm_representation.transpose())
transition = transition.dot(stack_lstm_representation)
print 'transition', transition.eval({stack_input:syn_stack_input, target_input:syn_target_input}).shape

# Decode words
decoded_words = target_word_decoding_weights.link(target_lstm.h)
print 'decoded_words', decoded_words.eval({stack_input:syn_stack_input, target_input:syn_target_input}).shape

# Compute seq-seq loss
decoding_loss = T.nnet.categorical_crossentropy(decoded_words, target_output).mean()
'''

In [ ]:
'''
Update parameters using ADAM
'''
updates_action=LearningMethod(clip=5.0).get_updates('adam', action_prediction_loss, action_params)

#updates_action = Optimizer().adam(
#                action_prediction_loss,
#                action_params,
#        )

In [ ]:
'''
Update parameters using ADAM
'''
updates_seq_seq = LearningMethod(clip=5.0).get_updates('adam', decoding_loss, seq_seq_params)
#updates_seq_eq = Optimizer().adam(
#                decoding_loss,
#                seq_seq_params,
#        )

In [ ]:
f_train_action = theano.function(
    inputs=[stack_input, forward_context_input, action_prediction],
    outputs=action_prediction_loss,
    updates=updates_action
)

In [ ]:
f_train_seq_seq = theano.function(
    inputs=[stack_input, target_input, target_output],
    outputs=decoding_loss,
    updates=updates_seq_seq
)

In [ ]:
f_get_action = theano.function(
    inputs=[stack_input, forward_context_input],
    outputs=prediction,
)

In [ ]:
for ind, sentence in enumerate(sentences):
    action_losses = []
    seq_seq_losses = []
    for state in sentence[2:]:
        word = state[0]
        action = state[1]
        stack_state = state[2].strip().split()
        forward_context = state[3].strip().split()
        translation = state[4].strip().split()
        stack_words = [source_word2ind[word] for word in stack_state]
        forward_context_words = [source_word2ind[word] for word in forward_context][::-1]
        action = 1.0 if action == 'T' else 0.0
        translation_words = [target_word2ind['<s>']] + [target_word2ind[word] for word in translation] + [target_word2ind['</s>']]
        action_loss = f_train_action(stack_words, forward_context_words, action)
        #seq_seq_loss = f_train_seq_seq(stack_words, translation_words[:-1], translation_words[1:])
        action_losses.append(action_loss)
        #seq_seq_losses.append(seq_seq_loss)
    print 'Sentence %d out of %d action loss %f ' % (ind, len(sentences), np.mean(action_losses))
    #print 'Sentence %d out of %d seq-seq loss %f ' % (ind, len(sentences), np.mean(seq_seq_losses))

In [151]:
print translation.strip().split()

AttributeError: 'list' object has no attribute 'strip'

In [25]:
translation_words[1:]

[71181, 71179]

3